### Crop the images

In [4]:
import os
from osgeo import gdal, ogr
from shapely.geometry import box
import geopandas as gpd

In [11]:
tif_path = '../data/actual_data/2023-02-23-bakonyszucs_im_cropped.tif'
shp_path = '../data/actual_data/Pont.shp'
out_folder ='../data/distributed'


In [12]:
num_rows = 1
num_cols = 2

In [17]:
ds = gdal.Open(tif_path)
width = ds.RasterXSize
height = ds.RasterYSize

gdf = gpd.read_file(shp_path)
#gdf['geometry'] = gdf['geometry'].centroid

# Filter out None geometries
gdf = gdf[gdf.geometry.notnull()]

part_width = width // num_cols
part_height = height // num_rows

for i in range(num_rows):
    for j in range(num_cols):
        #calculate bounding box coordinates
        xmin = j * part_width
        ymin = i * part_height
        xmax = (j + 1) * part_width
        ymax = (i + 1) * part_height
        
        #create a bounding box geometry
        bbox = box(xmin, ymin, xmax, ymax)
        
        #crop TIFF image
        output_tif = f'output_part_{i}_{j}.tif'
        gdal.Translate(os.path.join(out_folder,output_tif), ds, srcWin=[xmin, ymin, part_width, part_height])
        
        #crop shapefile
        #cropped_gdf = gdf[gdf.geometry.intersects(bbox)]
        #cropped_gdf = gdf[gdf.geometry.apply(lambda x: x.within(bbox))]
        #output_shp = f'output_part_{i}_{j}.shp'
        #cropped_gdf.to_file(os.path.join(out_folder,output_shp),driver='ESRI Shapefile')

/Users/banoczymartin/OE Projects/Mapping Upper Canopy Level/venvs_can/canopysegmentation/lib/python3.9/site-packages/geopandas/io/file.py:633: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)
/Users/banoczymartin/OE Projects/Mapping Upper Canopy Level/venvs_can/canopysegmentation/lib/python3.9/site-packages/geopandas/io/file.py:633: UserWarning: You are attempting to write an empty DataFrame to file. For some drivers, this operation may fail.
  _to_file_fiona(df, filename, driver, schema, crs, mode, **kwargs)


### Convert SHP to PNG

In [35]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt

def convert_PNGtoSHP(shp_path,png_path, point_size=1, dpi = 300,bg_color='black',fg_color='white', fig_size=(50,50)):
    '''
    returns the point as well as a list of tuples
    '''
    
    gdf = gpd.read_file(shp_path)

    xmin, ymin, xmax, ymax = gdf.total_bounds

    #plot points as white boxes on a black background
    fig, ax = plt.subplots(figsize=fig_size)
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_facecolor(bg_color)
    ax.scatter(gdf.geometry.x, gdf.geometry.y, s=point_size, color=fg_color) #point size

    #save the plot as a PNG image
    plt.savefig(png_path, bbox_inches='tight', pad_inches=0, facecolor='black', dpi=dpi)
    plt.close()
    
    #points = [(p.x, p.y) for p in gdf.geometry]
    return list(zip(gdf.geometry.x,gdf.geometry.y))#points (X, Y)
    
shp_path = '../data/actual_data/Pont.shp'
out_path = '../data/actual_data/out.png'
points = convert_PNGtoSHP(shp_path,out_path, fig_size=(10,10))
print(points)

[(549313.53553259, 221100.371878735), (549312.6498544463, 221111.39365119016), (549294.9854959132, 221106.03037798652), (549304.5311381288, 221122.12019759743), (549320.7931729344, 221098.354500741), (549323.6962290722, 221103.3733435554), (549332.4053974855, 221097.91166166915), (549333.586301677, 221104.11140867518), (549361.140732815, 221102.09403068115), (549347.51113027, 221118.67589370522), (549347.806356318, 221111.34444684887), (549340.1796834137, 221105.58753891473), (549349.528508264, 221104.55424774703), (549342.6399004796, 221093.97531436372), (549363.010497785, 221116.31408532197), (549308.8365179942, 221128.66437499272), (549335.1608405992, 221133.43719610054), (549339.7368443417, 221128.9103966993), (549327.8293937428, 221127.87710553163), (549295.1577111079, 221113.75545957344), (549299.044854072, 221129.35323577118), (549288.7119423953, 221114.09988996267), (549320.7439685932, 221098.354500741), (nan, nan), (549354.317315141, 221126.48480276036), (549317.8643463538, 22

In [36]:
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

# Load the PNG image and extract information about the white points
# For simplicity, let's assume you already have the coordinates of the white points
white_points = points#[(x1, y1), (x2, y2), ...]  # Replace with actual coordinates

# Create a list of Point geometries
point_geometries = [Point(x, y) for x, y in white_points]

# Create a GeoDataFrame with the point geometries
gdf_points = gpd.GeoDataFrame(geometry=point_geometries, crs='EPSG:4326')  # Use appropriate CRS

# Define output shapefile path
output_shp = '../data/actual_data/output_points.shp'

# Save the GeoDataFrame to a shapefile
gdf_points.to_file(output_shp)

In [22]:
import os
import rasterio
from rasterio.features import geometry_mask
from shapely.geometry import Point, box
import geopandas as gpd

# Load shapefile
gdf = gpd.read_file(shp_path)

# Define number of rows and columns to divide the shapefile
num_rows = 1
num_cols = 1

# Calculate extent of each part
xmin, ymin, xmax, ymax = gdf.total_bounds
part_width = (xmax - xmin) / num_cols
part_height = (ymax - ymin) / num_rows

# Define output folder
output_folder = '../data'

# Define resolution for rasterization
res = 0.01  # Adjust as needed

# Define size of points in pixels
point_size = 50  # Adjust as needed

for i in range(num_rows):
    for j in range(num_cols):
        # Calculate bounding box coordinates for the current part
        part_xmin = xmin + j * part_width
        part_ymin = ymin + i * part_height
        part_xmax = xmin + (j + 1) * part_width
        part_ymax = ymin + (i + 1) * part_height
        
        # Filter the shapefile to include only geometries within the current part
        bbox = box(part_xmin, part_ymin, part_xmax, part_ymax)
        cropped_gdf = gdf[gdf.geometry.intersects(bbox)]
        
        # Define output PNG file path for the current part
        output_png = os.path.join(output_folder, f'output_part_{i}_{j}.png')
        
        # Create a blank raster image for the current part
        width = int((part_xmax - part_xmin) / res)
        height = int((part_ymax - part_ymin) / res)
        with rasterio.open(
                output_png,
                'w',
                driver='PNG',
                width=width,
                height=height,
                count=1,
                dtype=rasterio.uint8,
                transform=rasterio.Affine(res, 0, part_xmin, 0, -res, part_ymax)) as dst:
            # Rasterize points onto the raster grid as white squares
            for index, row in cropped_gdf.iterrows():
                point = row.geometry
                if isinstance(point, Point):
                    x, y = point.xy
                    x, y = x[0], y[0]
                    # Convert point coordinates to pixel coordinates
                    px, py = dst.index(x, y)
                    # Calculate bounding box of the square around the point
                    square = box(px - point_size/2, py - point_size/2, px + point_size/2, py + point_size/2)
                    # Rasterize the square as white
                    mask = geometry_mask([square], out_shape=(height, width), transform=dst.transform, invert=True)
                    dst.write(mask.astype(rasterio.uint8) * 255, 1)


In [23]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt

# Load shapefile
gdf = gpd.read_file(shp_path)

# Define number of rows and columns to divide the shapefile
num_rows = 1
num_cols = 1

# Calculate extent of each part
xmin, ymin, xmax, ymax = gdf.total_bounds
part_width = (xmax - xmin) / num_cols
part_height = (ymax - ymin) / num_rows

# Define output folder
output_folder = '../data'

# Define size of points in pixels
point_size = 1  # Adjust as needed

for i in range(num_rows):
    for j in range(num_cols):
        # Calculate bounding box coordinates for the current part
        part_xmin = xmin + j * part_width
        part_ymin = ymin + i * part_height
        part_xmax = xmin + (j + 1) * part_width
        part_ymax = ymin + (i + 1) * part_height
        
        # Filter the shapefile to include only geometries within the current part
        bbox = [part_xmin, part_ymin, part_xmax, part_ymax]
        cropped_gdf = gdf.cx[part_xmin:part_xmax, part_ymin:part_ymax]
        
        # Plot points as white boxes on a black background
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xlim(part_xmin, part_xmax)
        ax.set_ylim(part_ymin, part_ymax)
        ax.set_facecolor('black')
        ax.scatter(cropped_gdf.geometry.x, cropped_gdf.geometry.y, s=point_size, color='white')
        
        # Save the plot as a PNG image
        output_png = os.path.join(output_folder, f'output_part_{i}_{j}.png')
        plt.savefig(output_png, bbox_inches='tight', pad_inches=0, facecolor='black', dpi=300)
        plt.close()


** copy shp params **

In [41]:
output_shp = 'output_shapefile.shp'

input_ds = ogr.Open(shp_path)
if input_ds is None:
    print(f"Could not open input shapefile: {shp_path}")
    exit()
    
# Get the input layer
input_layer = input_ds.GetLayer()

# Create a new shapefile for writing
driver = ogr.GetDriverByName('ESRI Shapefile')
if os.path.exists(output_shp):
    driver.DeleteDataSource(output_shp)
output_ds = driver.CreateDataSource(output_shp)
output_layer = output_ds.CreateLayer(input_layer.GetName(), input_layer.GetSpatialRef(), input_layer.GetGeomType())

# Define the fields and add them to the output layer
field_def_list = []
for i in range(input_layer.GetLayerDefn().GetFieldCount()):
    field_def = input_layer.GetLayerDefn().GetFieldDefn(i)
    field_def_list.append(field_def)
    output_layer.CreateField(field_def)

# Copy features from input layer to output layer with modified geometry (e.g., move features)
for feature in input_layer:
    geom = feature.GetGeometryRef()
    # Modify the geometry here (e.g., move the feature)
    # Example: geom.SetPoint(0, NEW_X, NEW_Y)  # Move the first point of the geometry
    new_feature = ogr.Feature(output_layer.GetLayerDefn())
    new_feature.SetGeometry(geom)
    for i, field_def in enumerate(field_def_list):
        new_feature.SetField(i, feature.GetField(i))
    output_layer.CreateFeature(new_feature)
    new_feature = None

# Close the shapefiles
input_ds = None
output_ds = None